In [125]:
import pickle
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [126]:
with open('similar_slide.pkl', 'rb') as f:
    data = pickle.load(f)

In [127]:
sim = {}
droped = set()
for i,j in data:
    if i in sim or j in sim:
        if i in sim:
            if j in sim[i]:
                continue
            else:
                sim[i].append(j)
        else:
            if i in sim[j]:
                continue
            else:
                sim[j].append(i)
        droped.add(i)
        droped.add(j)
    else:
        if j in droped or i in droped:
            continue
        sim[i] = [j]

In [128]:
with open('similar_slide_dict.pkl', 'wb') as f:
    pickle.dump(sim, f)

In [132]:
df = pd.read_csv("train_w_tile_number.csv")
df = df.set_index('image_id')
ndf = df.copy()

In [130]:
with open('slide_has_less_tiles.pkl', 'rb') as f:
    data = pickle.load(f)

In [133]:
# ndf = ndf.drop(data)
len(ndf)

10615

In [134]:
count = 0
for key in sim:
    for name in sim[key]:
        ndf = ndf.drop(name)
        count += 1

In [135]:
len(ndf), count

(10537, 78)

In [136]:
ndf_rad = ndf.loc[ndf.data_provider == "radboud"]
ndf_kar = ndf.loc[ndf.data_provider == "karolinska"]

In [137]:
ndf_rad = ndf_rad.reset_index()
ndf_kar = ndf_kar.reset_index()

In [138]:
len(ndf_rad), len(ndf_kar)

(5084, 5453)

In [139]:
nfolds = 4
SEED = 2020
## stratified KFold class that can preserve the sample percentatage in each fold. 
splits_rad = StratifiedKFold(n_splits=nfolds, random_state=SEED, shuffle=True) 
splits_rad = list(splits_rad.split(ndf_rad,ndf_rad.isup_grade)) ## list[(fold1_train_idx, fold1_test_idx), (fold2_train_idx, fold2_test_idx), ...]

## mark each sample to the fold, in which the sample serve as a test case
folds_splits = np.zeros(len(ndf_rad)).astype(np.int) ## [0, 0, ...] with the number of cases
for i in range(nfolds): 
    folds_splits[splits_rad[i][1]] = i

ndf_rad['split'] = folds_splits ## add the K fold assignment column
ndf_rad = ndf_rad.set_index('image_id')
ndf_rad.head(10)

,data_provider,isup_grade,gleason_score,tile_number,split
image_id,,,,,
0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4,26,0
004dd32d9cd167d9cc31c13b704498af,radboud,1,3+3,32,1
0068d4c7529e34fd4c9da863ce01a161,radboud,3,4+3,9,2
006f6aa35a78965c92fffd1fbd53a058,radboud,3,4+3,6,0
007433133235efc27a39f11df6940829,radboud,0,negative,18,3
0076bcb66e46fb485f5ba432b9a1fe8a,radboud,3,4+3,36,1
008069b542b0439ed69b194674051964,radboud,4,4+4,27,1
00928370e2dfeb8a507667ef1d4efcbb,radboud,5,4+5,79,3
00951a7fad040bf7e90f32e81fc0746f,radboud,1,3+3,32,2


In [140]:
nfolds = 4
SEED = 2020
## stratified KFold class that can preserve the sample percentatage in each fold. 
splits_kar = StratifiedKFold(n_splits=nfolds, random_state=SEED, shuffle=True) 
splits_kar = list(splits_kar.split(ndf_kar,ndf_kar.isup_grade)) ## list[(fold1_train_idx, fold1_test_idx), (fold2_train_idx, fold2_test_idx), ...]

## mark each sample to the fold, in which the sample serve as a test case
folds_splits = np.zeros(len(ndf_kar)).astype(np.int) ## [0, 0, ...] with the number of cases
for i in range(nfolds): 
    folds_splits[splits_kar[i][1]] = i

ndf_kar['split'] = folds_splits ## add the K fold assignment column
ndf_kar = ndf_kar.set_index('image_id')
ndf_kar.head(10)

,data_provider,isup_grade,gleason_score,tile_number,split
image_id,,,,,
0005f7aaab2800f6170c399693a96917,karolinska,0,0+0,26,3
000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0,12,2
001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4,37,2
001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0,62,2
002a4db09dad406c85505a00fb6f6144,karolinska,0,0+0,31,1
003046e27c8ead3e3db155780dc5498e,karolinska,1,3+3,35,2
0032bfa835ce0f43a92ae0bbab6871cb,karolinska,1,3+3,17,3
003a91841da04a5a31f808fb5c21538a,karolinska,1,3+3,11,1
003d4dd6bd61221ebc0bfb9350db333f,karolinska,1,3+3,32,2


In [141]:
len(ndf_rad), len(ndf_kar)

(5084, 5453)

In [142]:
for key in sim:
    if key in ndf_rad.index:
        for name in sim[key]:
            split = ndf_rad.loc[key, 'split']
            ndf_rad = ndf_rad.append(df.loc[name])
            ndf_rad.loc[name, "split"] = int(split)
    else:
        for name in sim[key]:
            split = ndf_kar.loc[key, 'split']
            ndf_kar = ndf_kar.append(df.loc[name])
            ndf_kar.loc[name, "split"] = int(split)

In [143]:
len(ndf_rad), len(ndf_kar)

(5160, 5455)

In [144]:
ndf_rad.to_csv("radboud_4_fold_train.csv")
ndf_kar.to_csv("karolinska_4_fold_train.csv")

In [145]:
ndf_all = pd.concat([ndf_rad, ndf_kar])

In [157]:
from sklearn.utils import shuffle
for i in range(10000):
    ndf_all = shuffle(ndf_all)
#     temp = shuffle(temp)

In [160]:
ndf_all.to_csv("4_fold_whole_train.csv")

In [159]:
ndf_all.head(20)

,data_provider,isup_grade,gleason_score,tile_number,split
image_id,,,,,
9716b5d2e13a28be3dc836134153720c,radboud,4,4+4,7,1.0
0954f0bb123cf3be8b00c9c989dcf3d6,karolinska,0,0+0,43,1.0
d861180850ca54282b38c782734a7dc7,karolinska,0,0+0,39,0.0
459154b1334d073cfef8af95362a7051,karolinska,0,0+0,34,1.0
27489a7f1972687cccb1258116ff7f10,karolinska,0,0+0,24,0.0
fb5fd7ab15e699998a30b36fe169431b,radboud,3,4+3,18,1.0
06f714d78f6f5c545cd149cf71c7c1c1,karolinska,0,0+0,68,2.0
f40c17b3402d3937434e08f419ebb3d9,radboud,4,3+5,32,3.0
746cfe9ab71459d51eb0fbdb72348ae8,radboud,5,5+4,20,3.0
